In [2]:
import pandas as pd
import joblib
import os
import sys

notebook_dir = os.getcwd() 
project_root = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(project_root)

from Database.database_manager import DatabaseManager
from Models.Functional_Classes.Glicko2.GlickoCalculator import GlickoCalculator
from Models.Functional_Classes.Glicko2.GlickoFighter import GlickoFighter
from Elo_System.Cross_Functions.create_features import prep_features
from utils.ranking_model import ranking_model


In [3]:
MODEL_PATH = './Performance_Vector_Modal/dominance_modal.pkl'
performance_modal = joblib.load(MODEL_PATH)

glicko_calculator = GlickoCalculator()

In [4]:
with DatabaseManager('../Database/fighters.db') as db:
    all_fights = db.get_fights()
    all_fighters = db.get_fighters()

all_fights = pd.DataFrame(all_fights)

In [5]:
glicko_fighters= {
    fighter.id : GlickoFighter(fighter.elo_rating, fighter.rating_deviation, fighter.rating_volatility)
    for fighter in all_fighters
}
print(glicko_fighters)

{1: GlickoFighter rating: 1500.0, rating_deviation: 350.0 volatility: 0.06, 2: GlickoFighter rating: 1500.0, rating_deviation: 350.0 volatility: 0.06, 3: GlickoFighter rating: 1500.0, rating_deviation: 350.0 volatility: 0.06, 4: GlickoFighter rating: 1500.0, rating_deviation: 350.0 volatility: 0.06, 5: GlickoFighter rating: 1500.0, rating_deviation: 350.0 volatility: 0.06, 6: GlickoFighter rating: 1500.0, rating_deviation: 350.0 volatility: 0.06, 7: GlickoFighter rating: 1500.0, rating_deviation: 350.0 volatility: 0.06, 8: GlickoFighter rating: 1500.0, rating_deviation: 350.0 volatility: 0.06, 9: GlickoFighter rating: 1500.0, rating_deviation: 350.0 volatility: 0.06, 10: GlickoFighter rating: 1500.0, rating_deviation: 350.0 volatility: 0.06, 11: GlickoFighter rating: 1500.0, rating_deviation: 350.0 volatility: 0.06, 12: GlickoFighter rating: 1500.0, rating_deviation: 350.0 volatility: 0.06, 13: GlickoFighter rating: 1500.0, rating_deviation: 350.0 volatility: 0.06, 14: GlickoFighter ra

In [6]:
features = prep_features(all_fights)

dominance_scores = performance_modal.predict(features)
dominance_scores = pd.DataFrame(dominance_scores)

show_scores = pd.concat([features, dominance_scores], axis=1)
all_fights['dominance_score'] = dominance_scores

show_scores.head()

,sig_strike_differential,takedown_differential,knockdown_differential,sub_differential,total_sig_strikes,total_takedowns,total_knockdowns,total_submissions,final_round,final_time_seconds,win_method_clean_CNC,win_method_clean_DQ,win_method_clean_KO/TKO,win_method_clean_M-DEC,win_method_clean_Other,win_method_clean_Overturned,win_method_clean_S-DEC,win_method_clean_SUB,win_method_clean_U-DEC,0
0,-11,0,0,0,27,0,0,0,2,18,1,0,0,0,0,0,0,0,0,0.970353
1,11,0,1,0,21,0,1,0,1,71,0,0,1,0,0,0,0,0,0,2.276725
2,0,0,1,0,2,0,1,0,1,22,0,0,1,0,0,0,0,0,0,2.276725
3,19,0,0,0,77,0,2,0,2,183,0,0,1,0,0,0,0,0,0,1.639512
4,-6,2,0,1,116,2,0,1,3,300,0,0,0,0,0,0,1,0,0,0.873416


In [7]:
for index, fight_row in all_fights.iterrows():
    red_id = fight_row['red_fighter_id']
    blue_id = fight_row['blue_fighter_id']
    winner_id = fight_row['winner_id']

    if pd.isna(red_id) or pd.isna(blue_id) or red_id not in glicko_fighters or blue_id not in glicko_fighters:
        continue

    red_fighter = glicko_fighters[red_id]
    blue_fighter = glicko_fighters[blue_id]

    dominance_score = fight_row['dominance_score']

    if winner_id == red_id:
        outcome_red = min(1.0, dominance_score / 2.0)
        outcome_blue = 1- outcome_red
    elif winner_id == blue_id:
        outcome_blue = min(1.0, dominance_score / 2.0)
        outcome_red = 1- outcome_blue
    else:
        outcome_red = 0.5
        outcome_blue = 0.5

    red_snapshot = GlickoFighter(red_fighter.rating, red_fighter.volatility,red_fighter.rating_deviation )
    blue_snapshot = GlickoFighter(blue_fighter.rating,blue_fighter.volatility, blue_fighter.rating_deviation )

    glicko_calculator.rate_1vs1(red_fighter, blue_snapshot, outcome_red)
    glicko_calculator.rate_1vs1(blue_fighter, red_snapshot, outcome_blue)




In [8]:

final_rankings = ranking_model(glicko_fighters, all_fighters, all_fights)
final_rankings.head(25)


,#,id,Name,Rating,RD,Quality Score
2169,1,3674,Khamzat Chimaev,1856.618221,133.199431,1585.481973
1604,2,2727,Ilia Topuria,1846.563695,130.026992,1574.456501
1017,3,1661,Francis Ngannou,1844.436343,102.658629,1556.621014
1736,4,2947,Islam Makhachev,1781.733417,100.411149,1550.175088
56,5,95,Kamaru Usman,1738.590595,85.336062,1547.039549
2113,6,3582,Jon Jones,1745.876920,79.083075,1546.891378
398,7,675,Amanda Nunes,1797.580340,94.121219,1543.913361
152,8,262,Carlos Prates,1838.189130,147.575368,1538.059819
1477,9,2480,Jailton Almeida,1814.145963,126.431845,1530.274072
1072,10,1773,Max Holloway,1685.805098,73.431453,1528.489064


In [9]:
glicko_fighters_temp = glicko_fighters.copy()

with DatabaseManager('../Database/fighters.db') as db:
    db.update_all_fighters(glicko_fighters)

In [10]:
print(glicko_fighters)

{1: GlickoFighter rating: 1421.6586669350247, rating_deviation: 176.2143231041136 volatility: 0.0599938394746137, 2: GlickoFighter rating: 1432.4645225084273, rating_deviation: 155.39689871444077 volatility: 0.059999023605418096, 3: GlickoFighter rating: 1517.7416874841258, rating_deviation: 145.78313079469947 volatility: 0.059989150765745715, 4: GlickoFighter rating: 1500.0, rating_deviation: 350.0 volatility: 0.06, 5: GlickoFighter rating: 1500.0, rating_deviation: 350.0 volatility: 0.06, 6: GlickoFighter rating: 1500.0, rating_deviation: 350.0 volatility: 0.06, 7: GlickoFighter rating: 1513.5726182040642, rating_deviation: 109.06821793332676 volatility: 0.0599786416703554, 8: GlickoFighter rating: 1233.6101537907598, rating_deviation: 222.96830678835005 volatility: 0.05999852746290063, 9: GlickoFighter rating: 1500.0, rating_deviation: 350.0 volatility: 0.06, 10: GlickoFighter rating: 1500.0, rating_deviation: 350.0 volatility: 0.06, 11: GlickoFighter rating: 1500.0, rating_deviatio

In [11]:
#with DatabaseManager('../Database/fighters.db') as db:
#    db.reset_fighter_ratings()
